# Python中階-第一支爬蟲程式


## 中正大學資管系 (20181021) 大綱

這部分會說明如何完成你的第一支爬蟲程式，接著將這支程式結構化

+ 起手式
+ 分析網站與內容
+ 爬你所見
+ 爬你所想
+ 結構化程式

## 起手式
+ `import requests` 這個套件
+ 建立一個 `resp` 變數去儲存 `requests` 抓到的網頁內容
+ `resp` 的變數除了內容外也會回傳一個 html 狀態碼

通常這一部成功代表網頁並未針對爬蟲做任何阻擋，有些網站會基於某些理由讓別人爬他的網站，例如: 購物網站(蝦皮)，或是擁有大量資料的網站，例如股票訊息，這些網站防止爬蟲理由可能基於避免大量爬蟲而影響真正使用者的使用感受(速度變慢等等)

但這邊我們都不先談這些複雜的破解阻擋方法(可能某些也無法破解)

In [1]:
import requests

url = 'https://www.ptt.cc/bbs/movie/index.html'
resp = requests.get(url)

In [10]:
resp, resp.status_code

(<Response [200]>, 200)

## 分析網站內容與結構

網站的內容組成不外乎就是 html + css，這些構成的結構通常我們會稱為標籤（就是前者的語法，有這些語法才能形成類似表格、表單等等）
在爬蟲之前其實最困難的是理解這些結構。要分析這個結構通常會推薦使用 Google Chrome 瀏覽器的`開發人員功能`，開啟方法就是在網頁裡點選右鍵，找到`檢查功能`點選就會看到如下圖(版面調整可從檢查功能右上角選單做改變)，或是你也可以按 F12 快速鍵開啟

![](images/CHROME.png)

## 爬你所見

以 ptt 網頁版為例子，整個版面是在 `<body>..</body>` 標籤內，但實際讓每篇文章又被歸類在 `<div class='r-ent'></div>` 下懂了！所以如果我要爬文章，最簡單方法就是去找出所有標籤是 `<div class='r-ent'></div>` 的內容就好了對吧？

In [ ]:
from requests_html import HTML
html = HTML(html=resp.text)
post_entries = html.find('div.r-ent')
post_entries

post_entries 會使用一個 `list(串列)`儲存爬取頁面的所有的文章，印出來你看到的會是作者將這些 html, css 的檔案內容包裝成某個物件的型態(可以比對 `BeautifulSoup` 並未包裝)。但現在會有個疑問，除了看文件了解要怎樣取出物件型態的內容，還有什麼方法可以知道呢？

還記得基礎課程所教的 `dir()` ? 讓它派上用場吧 dir(post_entries[0]) 馬上可以看到他提供很多功能讓我們能再進一步取得每篇文章底下的訊息。但這時會發現，底下的內容也是一連串的元素，這些元素各自有標籤存放著訊息，繼續用`開發人員工具`看看？接著把 post_entries 內的值印出來確認是否一樣？

![#](images/CHROME2.png)

In [20]:
ent1 = post_entries[0] # 先試試第一片是否為公告
ent1.find('div.title', first=True).text

'[公告] 水桶公告 20181017'

首先為什麼 ent1.find() 要在第二個參數放 `first=True`? 印為這個套件作者在[文件上說明](https://html.python-requests.org/)，如果要爬取的是一個 [css selector](https://www.w3schools.com/cssref/css_selectors.asp) 那使用 `.find` 這個方法就需要加這個參數

如果是要爬取 `<div><title></div>` 下的超連結，那就需要透過 `css selector` 語法去往下搜尋

In [25]:
ent1.find('div.title > a', first=True).attrs['href']

'/bbs/movie/M.1539741182.A.7CD.html'

In [26]:
ent1.find('div.title a', first=True).attrs['href']

'/bbs/movie/M.1539741182.A.7CD.html'

## 爬你想要

當瞭解一個網站結構之後接著就可以開始嘗試寫邏輯把內容爬出，接著我們就來示範一下:
+ 首先會使用一個 `for loop` 將存在 `post_entries` 這個 list 的資料結構內容讀出來 
+ 接著會建立一個 context 的 dict，根據 **title, push, date, author** 做 key 存起來
+ 最後印出內容

In [27]:
for entry in post_entries:
    context = {
        'title': entry.find('div.title', first=True).text,
        'push': entry.find('div.nrec', first=True).text,
        'date': entry.find('div.date', first=True).text,
        'author': entry.find('div.author', first=True).text,
    }
    print(context)

{'title': '[公告] 水桶公告 20181017', 'push': '1', 'date': '10/17', 'author': 'VOT1077'}
{'title': '[新聞] 登陸月球50年《電影哆啦A夢 大雄的月球探', 'push': '1', 'date': '10/17', 'author': 'hoanbeh'}
{'title': 'Re: [贈票] 【極智對決】 台北贈票', 'push': '10', 'date': '10/17', 'author': 'rapsd520'}
{'title': '新聞文章請以新發文方式-V <Reewalker>', 'push': '', 'date': '10/17', 'author': '-'}
{'title': '[請益] 李小龍傳', 'push': '4', 'date': '10/17', 'author': 'hsinofkids'}
{'title': '[討論] 最經典的系列作有哪些', 'push': '3', 'date': '10/17', 'author': 'assggy'}
{'title': '[版規] 電影版版規 201808', 'push': '1', 'date': '8/28', 'author': 'VOT1077'}
{'title': '[公告] 電影版板規修訂說明', 'push': '', 'date': '8/28', 'author': 'VOT1077'}
{'title': '[公告] 關於特定影片負(好)雷討論', 'push': '33', 'date': '10/11', 'author': 'VOT1077'}


### 程式碼目前應該會長成這樣
```python
import requests
from requests_html import HTML

url = 'https://www.ptt.cc/bbs/movie/index.html'
resp = requests.get(url)
html = HTML(html=resp.text)
post_entries = html.find('div.r-ent')

for entry in post_entries:
    context = {
        'title': entry.find('div.title', first=True).text,
        'push': entry.find('div.nrec', first=True).text,
        'date': entry.find('div.date', first=True).text,
        'author': entry.find('div.author', first=True).text,
    }
    print(context)
```

## 結構化程式

到目前為止雖然成功完成了第一個爬蟲程式，但通常會認為要將程式做結構化的調整，意思是如果將程式碼從頭寫到尾不做結構化，通常許多類似功能會重複的出現，這不但為讓程式碼看起來冗長且維護起來不容易

因此我們先來看看哪些部分是有可能是可以作為修改的？

+ 找出可以作為變數的部分
+ 將可能重複使用的部分轉變成函式

看起來 `url` 這個變數與 `requests.get(url)` 這個方法在使用這支函式時都會使用到，而且會改變部分只有 `url` 那先試著把這邊做結構化動作 

In [29]:
def fetch(url):
    response = requests.get(url)
    return response

In [30]:
resp = fetch(url='https://www.ptt.cc/bbs/movie/index.html')
resp

<Response [200]>

成功完成第一步，接著我們來思考
+ 當取的內容之後會經過一層處理接著回傳給我們想要的資料結構，但有可能這部分未必是每次內容都要相同，或許把其升級為函式彈性也會更好些

In [39]:
def parser_article_meta(entry):
    context = {
        'title': entry.find('div.title', first=True).text,
        'push': entry.find('div.nrec', first=True).text,
        'date': entry.find('div.date', first=True).text,
        'author': entry.find('div.author', first=True).text,
    }
    return context

目前已經把兩個部分升級為函式，通常我們可能會把流程放在一個 `main()` 去進行處理

In [40]:
def main():
    resp = fetch(url='https://www.ptt.cc/bbs/movie/index.html')
    if resp.status_code == 200:
        html = HTML(html=resp.text)
        post_entries = html.find('div.r-ent')

        for entry in post_entries:
            meta = parser_article_meta(entry)
            print(meta)
    else:
        print(resp.status_code)

if __name__ == '__main__':
    main()

{'title': '[公告] 水桶公告 20181017', 'push': '2', 'date': '10/17', 'author': 'VOT1077'}
{'title': '[新聞] 登陸月球50年《電影哆啦A夢 大雄的月球探', 'push': '3', 'date': '10/17', 'author': 'hoanbeh'}
{'title': 'Re: [贈票] 【極智對決】 台北贈票', 'push': '14', 'date': '10/17', 'author': 'rapsd520'}
{'title': '新聞文章請以新發文方式-V <Reewalker>', 'push': '', 'date': '10/17', 'author': '-'}
{'title': '[請益] 李小龍傳', 'push': '4', 'date': '10/17', 'author': 'hsinofkids'}
{'title': '[討論] 最經典的系列作有哪些', 'push': '4', 'date': '10/17', 'author': 'assggy'}
{'title': '[新聞] 重建熱蘭遮城將投入135億 魏德聖：2024', 'push': '48', 'date': '10/17', 'author': 'purue'}
{'title': '[贈票] 極智對決 週六台北贈票', 'push': '', 'date': '10/17', 'author': 'WAV'}
{'title': '[情報] 2018 亞太銀幕獎 入圍名單', 'push': '', 'date': '10/17', 'author': 'qpr322'}
{'title': '[問片] 孕婦車禍流產，找人復仇的血腥片', 'push': '1', 'date': '10/17', 'author': 'shuffling'}
{'title': '[新聞] 華倫夫婦加入《安娜貝爾3》', 'push': '1', 'date': '10/17', 'author': 'shengchiu303'}
{'title': '[版規] 電影版版規 201808', 'push': '1', 'date': '8/28', 'author': 'VOT1

接著來看一起我們的程式目前會長這樣 

```python
#-*- coding: utf-8 -*-
import requests
from requests_html import HTML


def fetch(url):
    response = requests.get(url)
    return response


def parser_article_meta(entry):
    context = {
        'title': entry.find('div.title', first=True).text,
        'push': entry.find('div.nrec', first=True).text,
        'date': entry.find('div.date', first=True).text,
        'author': entry.find('div.author', first=True).text,
    }
    return context


def main():
    resp = fetch(url='https://www.ptt.cc/bbs/movie/index.html')
    if resp.status_code == 200:
        html = HTML(html=resp.text)
        post_entries = html.find('div.r-ent')

        for entry in post_entries:
            meta = parser_article_meta(entry)
            print(meta)
    else:
        print(resp.status_code)

if __name__ == '__main__':
    main()
```

把它存檔吧！你已經完成了第一個爬蟲程式了，有沒有好棒棒！